# Chroma

Chroma는 임베딩 벡터를 저장하기 위한 오픈소스 소프트웨어로, LLM(대규모 언어 모델) 앱 구축을 용이하게 하는 핵심 기능을 수행합니다. Chroma의 주요 특징은 다음과 같습니다   
   
- 임베딩 및 메타데이터 저장: 대규모의 임베딩 데이터와 이와 관련된 메타데이터를 효율적으로 저장할 수 있습니다.   
- 문서 및 쿼리 임베딩: 텍스트 데이터를 벡터 공간에 매핑하여 임베딩을 생성할 수 있으며, 이를 통해 검색 작업이 가능합니다.   
- 임베딩 검색: 사용자 쿼리에 기반하여 가장 관련성 높은 임베딩을 찾아내는 검색 기능을 제공합니다.   

In [5]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

loader = TextLoader('history.txt')
data = loader.load()

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=250, # 250 토큰씩 끊고
    chunk_overlap=50, # 겹치는 건 50씩
    encoding_name='cl100k_base'
)

texts = text_splitter.split_text(data[0].page_content) # 여기서 연산이 들어감
print("요소의 개수", len(texts))
print("texts[0]의 내용:", texts[0])

요소의 개수 6
texts[0]의 내용: 한국의 역사는 수천 년에 걸쳐 이어져 온 긴 여정 속에서 다양한 문화와 전통이 형성되고 발전해 왔습니다. 고조선에서 시작해 삼국 시대의 경쟁, 그리고 통일 신라와 고려를 거쳐 조선까지, 한반도는 많은 변화를 겪었습니다.

고조선은 기원전 2333년 단군왕검에 의해 세워졌다고 전해집니다. 이는 한국 역사상 최초의 국가로, 한민족의 시원이라 할 수 있습니다. 이후 기원전 1세기경에는 한반도와 만주 일대에서 여러 소국이 성장하며 삼한 시대로 접어듭니다.


In [6]:
embeddings_model = OpenAIEmbeddings() # OpenAI 임베딩을 사용하고,

db = Chroma.from_texts(
    texts, # 각각의 텍스트에 대하여 
    embeddings_model, # 임베딩을 해줄 건데
    collection_name = 'history', # 저장소 이름은 이걸로 해줄 거고
    persist_directory = './db/chromadb', # 저장은 여기에 할 거야
    collection_metadata = {'hnsw:space': 'cosine'}, # l2 is the default
)

db

In [7]:
query = '누가 한글을 창제했나요?'
docs = db.similarity_search(query)
print(docs[0].page_content)

조선은 1392년 이성계에 의해 건국되어, 1910년까지 이어졌습니다. 조선 초기에는 세종대왕이 한글을 창제하여 백성들의 문해율을 높이는 등 문화적, 과학적 성취가 이루어졌습니다. 그러나 조선 후기에는 내부적으로 실학의 발전과 함께 사회적 변화가 모색되었으나, 외부로부터의 압력은 점차 커져만 갔습니다.


## MMR (Maximum marginal relevance search)

유사성 점수 + 다양성 점수를 조합하여 각 문서의 최종 점수를 계산한 후에 이 최종 점수에 기반하여 문서를 선택하는 방법론   
   
MMR의 장점   
- 중복 제거: 이미 선택된 문서와 중복되는 결과를 줄일 수 있습니다.
- 다양성 향상: 검색 결과가 다양한 주제와 관점을 포함하도록 보장합니다.
- 유연성: λ 값을 조정하여 관련성과 다양성의 중요도를 조절할 수 있습니다.
   
MMR의 응용 분야   
- 문서 검색: 검색 결과에서 중복을 줄이고, 다양한 정보를 포함한 결과를 제공.
- 추천 시스템: 사용자의 이전 선택과 중복되지 않는 추천 제공.
- 문서 요약: 동일한 정보를 반복하지 않고 다양한 문장으로 요약 생성.
- 대화형 AI: 대화의 다양성과 관련성을 동시에 유지.

In [11]:
# Load data -> Text split

from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

loader = PyMuPDFLoader('323410_카카오뱅크_2023.pdf') # 불러오고
data = loader.load()
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000, 
    chunk_overlap=200,
    encoding_name='cl100k_base'
)

documents = text_splitter.split_documents(data)
len(documents)

# 여기까지는 기존 방식이랑 동일하다

9

In [12]:
documents

[Document(metadata={'source': '323410_카카오뱅크_2023.pdf', 'file_path': '323410_카카오뱅크_2023.pdf', 'page': 0, 'total_pages': 3, 'format': 'PDF 1.6', 'title': '', 'author': '교보', 'subject': '', 'keywords': '', 'creator': 'Microsoft® Word 2013', 'producer': 'Microsoft® Word 2013', 'creationDate': "D:20230811085350+09'00'", 'modDate': "D:20230811085401+09'00'", 'trapped': ''}, page_content='은행업  \n2023 년 2 분기 실적 Review 및 투자의견 점검 \n \n \n \n  \n \n \n \n \n Company Analysis\n \n \n카카오뱅크 323410 Aug 11, 2023\n \n높은 고객활동성이 보여준 성장!, 그리고 확장성 \n \nBuy  \n유지 \nTP  33,000 원  \n  상향 \n현재가(08/10) \n27,750 원 \n액면가(원) \n5,000 원 \n52주 최고가(보통주) \n33,150 원 \n52주 최저가(보통주) \n15,850 원 \nKOSPI (08/10) \n2,601.56p \nKOSDAQ (08/10) \n911.29p \n자본금 \n23,758 억원 \n시가총액 \n132,303 억원 \n발행주식수(보통주) \n47,677 만주 \n발행주식수(우선주) \n0 만주 \n평균거래량(60일) \n207.0 만주 \n평균거래대금(60일) \n572 억원 \n외국인지분(보통주) \n15.31% \n주요주주  \n \n카카오 \n27.17% \n한국투자증권 \n27.17% \n Price & Relative Performance\n \n \n주가수익률(%) \n1개월 \n6개월 \n12개월 \n절대

In [13]:
# Embedding -> Upload to Vectorstore
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings()
db2 = Chroma.from_documents(
    documents, 
    embeddings_model,
    collection_name = 'esg',
    persist_directory = './db/chromadb',
    collection_metadata = {'hnsw:space': 'cosine'}, # l2 is the default
)

db2


In [14]:
query = '카카오뱅크의 환경목표와 세부추진내용을 알려줄래?'
docs = db2.similarity_search(query)
print(len(docs))
print(docs[0].page_content)

4
은행업  
2023 년 2 분기 실적 Review 및 투자의견 점검 
 
 
 
  
 
 
[도표 37] 카카오뱅크 2023 년 2 분기 실적                                                (단위 : 십억원, %)  
 
 
2Q23 
1Q23 
QoQ 
2Q22 
YoY 
영업수익 
613.3  
560.5  
9.4  
370.8  
65.4  
이자수익 
494.6  
451.5  
9.5  
292.9  
68.9  
Fee수익 
47.9  
45.5  
5.3  
43.7  
9.6  
플랫폼수익 
18.0  
17.7  
1.7  
21.6  
-16.7  
기타영업수익 
52.8  
45.8  
N.A. 
12.6  
N.A. 
영업비용 
442.9  
373.1  
18.7  
249.5  
77.5  
이자비용 
240.4  
189.3  
27.0  
75.6  
218.0  
Fee & 플랫폼비용 
42.9  
47.7  
-10.1  
38.7  
10.9  
CD/ATM 비용 
16.2  
16.2  
0.0  
14.8  
9.5  
판매관리비 
107.3  
92.8  
15.6  
90.6  
18.4  
기타영업비용 
27.5  
27.1  
1.5  
23.8  
15.5  
충당금전입액(-) 
58.6  
51.0  
14.9  
46.9  
24.9  
영업이익 
111.8  
136.4  
-18.0  
74.4  
50.3  
영업외손익 
-3.5  
-1.2  
N.A. 
1.1  
N.A. 
법인세차감전이익 
108.3  
135.2  
-19.9  
75.5  
43.4  
법인세(-) 
26.3  
33.3  
-21.0  
18.5  
42.2  
당기순이익 
82.0  
101.9  
-19.5  
57.0  
43.9  
자료: 교보증권 리서치센터 
[도표 38] 카카오뱅크 총 고객수 및 MAU 추이 
 
 
자료: 교보증권 리서치센터 
[도표 39] 순이자마진 추이

In [15]:
print(docs[-1].page_content)

5,880.1 
부채총계 
10,986.3 
21,045.1 
23,853.0 
30,518.7 
34,933.8 
49,881.5 
54,326.7 
56,251.8 
자본금 
1,300.0 
1,825.5 
2,038.3 
2,375.8 
2,381.4 
2,381.4 
2,381.4 
2,381.4 
자본잉여금 
0.0 
0.0 
759.8 
2,956.2 
2,957.9 
2,957.9 
2,957.9 
2,957.9 
이익잉여금 
-141.2 
-127.5 
-13.8 
190.3 
453.4 
837.0 
1,362.5 
1,959.9 
자본조정 
-18.7 
-22.1 
4.9 
26.8 
19.6 
19.6 
19.6 
19.6 
기타포괄손익누계액 
0.3 
2.8 
7.8 
-27.8 
-116.7 
-116.7 
-116.7 
-116.7 
자기자본 
1,140.4 
1,678.7 
2,797.0 
5,521.4 
5,812.3 
6,195.9 
6,721.4 
7,318.8 
비지배지분 
0.0 
0.0 
0.0 
0.0 
0.0 
0.0 
0.0 
0.0 
지배지분 자기자본 
1,140.4 
1,678.7 
2,797.0 
5,521.4 
5,812.3 
6,195.9 
6,721.4 
7,318.8 
 
  
 
 
 
 
 
 
주요 투자지표 
 
 
 
 
 
 
 
 
12월 결산(원,배,%) 
 FY18 
 FY19 
 FY20 
 FY21 
 FY22 
 FY23E 
 FY24E 
 FY25E 
ROA(%) 
-0.2% 
0.1% 
0.5% 
0.7% 
0.7% 
0.8% 
0.9% 
1.0% 
ROE(%) 
-2.3% 
1.0% 
5.1% 
4.9% 
4.6% 
6.4% 
8.1% 
8.5% 
EPS(원) 
-31 
34 
277 
430 
552 
805 
1,103 
1,254 
BPS(원) 
1,687 
4,098 
6,828 
11,620 
12,203 
13,009 
14,112 
15,366 
PER(배) 
n.a 

In [ ]:
mmr_docs = db2.max_marginal_relevance_search(query, k=4, fetch_k=10)
print(len(mmr_docs))
print(mmr_docs[0].page_content)

In [ ]:
print(mmr_docs[-1].page_content)

## vectorstore에 메타데이터를 추가

In [16]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="LangChain은 대규모 언어 모델(LLM)을 사용하는 애플리케이션을 개발하기 위한 프레임워크입니다.",
        metadata={
            "title": "LangChain 소개",
            "author": "AI 개발자",
            "url": "http://example.com/langchain-intro"
        }
    ),
    Document(
        page_content="벡터 데이터베이스는 고차원 벡터를 효율적으로 저장하고 검색하는 데 특화된 데이터베이스 시스템입니다.",
        metadata={
            "title": "벡터 데이터베이스 개요",
            "author": "데이터 과학자",
            "url": "http://example.com/vector-db-overview"
        }
    )
    # 추가 문서들...
]

In [17]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

# OpenAI 임베딩 모델 초기화
embedding_model = OpenAIEmbeddings()

# Chroma 벡터 스토어에 문서와 메타데이터 저장
vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embedding_model,
    persist_directory="./chroma_db"  # 벡터 스토어를 디스크에 저장
)

In [20]:
query = "데이터베이스란 무엇인가요?"
results = vectorstore.similarity_search(query, k=1)

for doc in results:
    print(f"내용: {doc.page_content}")
    print(f"제목: {doc.metadata['title']}")
    print(f"저자: {doc.metadata['author']}")
    print(f"URL: {doc.metadata['url']}")
    print("---")

내용: 벡터 데이터베이스는 고차원 벡터를 효율적으로 저장하고 검색하는 데 특화된 데이터베이스 시스템입니다.
제목: 벡터 데이터베이스 개요
저자: 데이터 과학자
URL: http://example.com/vector-db-overview
---


# FAISS (Facebook AI Similarity Search)

유사도 기준   
- l2: 유클리디안 거리를 기준으로 함   
- ip: 내적을 통해 방향성을 평가함
- cosine: 코사인 유사도 기반


In [22]:
# 벡터스토어 db 인스턴스를 생성
from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain_community.embeddings import HuggingFaceEmbeddings

# 임베딩 모델 초기화하고
embeddings_model = HuggingFaceEmbeddings(
    model_name='jhgan/ko-sbert-nli',
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings':True},
)


vectorstore = FAISS.from_documents(documents,
                                   embedding = embeddings_model,
                                   distance_strategy = DistanceStrategy.COSINE
                                  )
vectorstore

In [23]:
vectorstore.distance_strategy

<DistanceStrategy.COSINE: 'COSINE'>

In [24]:
query = '카카오뱅크가 중대성 평가를 통해 도출한 6가지 중대 주제는 무엇인가?'
docs = vectorstore.similarity_search(query)
print(len(docs))
print(docs[0].page_content)

2
LangChain은 대규모 언어 모델(LLM)을 사용하는 애플리케이션을 개발하기 위한 프레임워크입니다.


In [25]:
mmr_docs = vectorstore.max_marginal_relevance_search(query, k=4, fetch_k=10)
print(len(mmr_docs))
print(mmr_docs[0].page_content)


2
LangChain은 대규모 언어 모델(LLM)을 사용하는 애플리케이션을 개발하기 위한 프레임워크입니다.


In [26]:
# save db
vectorstore.save_local('./db/faiss')

In [28]:
# load db
db3 = FAISS.load_local('./db/faiss', embeddings_model, allow_dangerous_deserialization= True)
